In [ ]:
# Importar dependencias
from s3fs.core import S3FileSystem
import pandas as pd
import os

In [ ]:
# Agregar credenciales
access = "capacitacion" # reemplazar por credenciales
secret = "capacitacion" # reemplazar por credenciales
s3 = S3FileSystem(
    key=access,
    secret=secret,
    endpoint_url="http://10.91.160.65:9000"
)

In [ ]:
# Exploremos nuestra conexión
buckets = s3.ls('')

# Imprimir los buckets a los que tenemos acceso
for bucket in buckets:
    print(bucket)

In [ ]:
# Veamos los archivos dentro de un bucket
files = s3.ls('capacitacion-lago-datos')

# Imprimir archivos dentro del bucket
for file in files:
    print(file)

Creamos un dataframe de juguete, para escribirlo en minio

In [ ]:
# Columnas de juguete
mi_nombre = "kl"

x = [i for i in range(10)]
y = [i + 3 for i in range(10)]

# Escribir archivo con pandas
df = pd.DataFrame({"x": x, "y": y})
df.to_parquet(f"../data/df_ejemplo_{mi_nombre}.parquet")
df

In [ ]:
# Escribir en minio el archivo parquet
bucket = "capacitacion-lago-datos"
file = f"df_ejemplo_{mi_nombre}.parquet"
source_folder = "../data"

minio_path = f"{bucket}/data-ejemplo/{file}"
local_path = f"{source_folder}/{file}"
s3.put_file(rpath=minio_path,lpath=local_path)

Replicamos varias veces el archivo en nuestro directorio local

In [ ]:
# Contiene el archivo creado manualmente
original_file =  pd.read_parquet(local_path)

# Vamos a escribir varias veces el mismo archivo
for i in range(10):
    print(f"Escribiendo archivo {i}")
    original_file.to_parquet(f"../data/replica_{i}_{mi_nombre}.parquet")     

In [ ]:
# Escribimos todo en minio
parquet_files =  os.listdir("../data/")
parquet_files = [file for file in parquet_files if "parquet" in file ]

bucket = "capacitacion-lago-datos"
for file in parquet_files:
    print(f"--------- Escribiendo en S3 archivo {file}")
    minio_path = f"{bucket}/data-ejemplo/{file}"
    file_path = f"../data/{file}"
    s3.put_file(lpath=file_path,rpath=minio_path)


In [ ]:
# Revisemos que todo haya quedado bien escrito
files = s3.ls('capacitacion-lago-datos/data-ejemplo')

# Imprimir archivos dentro del bucket
for file in files:
    print(file)


In [ ]:
# Leer un archivo desde minio
minio_path = f"s3://capacitacion-lago-datos/data-ejemplo/replica_1_{mi_nombre}.parquet"
parquet_example = pd.read_parquet(minio_path, filesystem=s3)
parquet_example